This Jupyter Notebook was inspired by $\textbf{Problem 3.18}$ in $$``\text{An Introduction to Computer Simulation Methods
(Applications to Physical System)}''$$
by Harvey Gould, Jan Tobochnik, and Wolfgang Christian.

A magnetic dipole with dipole moment $\mathbf{p} = |\mathbf{p}|\hat{\textbf{p}}$ produces the following magnetic field:
$$\textbf{B}({\textbf{r}}) = \frac{\mu_0 p}{4\pi \epsilon_0r^3} \left[(3\hat{\textbf{p}}\cdot \hat{\textbf{r}})\hat{\textbf{r}} - \hat{\textbf{p}} \right].$$

Although the trajectory of a charged particle in constant electric and magnetic fields can be solved analytically, the trajectories in the presence of dipole fields cannot. For this case, we must fall back on numerical methods, such as the one implemented in this notebook.

In [1]:
# We begin by importing the necessary packages.
# plotly's graphing packages are used to display interactive 3D plots for the particle's trajectory.

import numpy as np
from scipy.integrate import solve_ivp

import pandas as pd
import plotly.express as px

In [2]:
# The next step is to define all the physical parameters for the experiment (simulation).

E = [0, 0, 0]   # I'm considering there to be no electrical fields in this situation.
p = np.array([0, 0, 1]) # this defines the dipole moment (unit) vector.

# for this experiment, I'm considering a hypothetical particle with charge 5E-16 Coulombs and mass 1E-29 kg.
q = 5E-16
m0 = 1E-29

# the field for this simulation is 1E-11 Teslas.
# the speed of light is also defined here (in SI units), since the relatively correct mass will be used to model the dynamics of the charged particle.
B0 = 1E-11
c = 3E8

The trajectory of the charged particle will be calculated using the Lorentz force law, in conjunction with Newton's second law:
$$\mathbf{F} = q(\mathbf{E} + \mathbf{v}\times \mathbf{B}),$$
$$\mathbf{a} = \frac{d\mathbf{v}}{dt} = \frac{1}{m}\mathbf{F}.$$

These two equations are solved component-wise, in the form of three coupled differential equations.

In [3]:
def DipoleField(t, S):
    '''DipoleField defines how the charged particle will respond to the magnetic dipole field it is placed in. The state vector S is defined by:
    S = [x, xDot, y, yDot, z, zDot].'''
    
    # I begin by storing the separation vector...
    r = np.array([S[0], S[2], S[4]])
    R = np.linalg.norm(r)
    r /= R

    # the next step is to store the velocity.
    v = np.array([S[1], S[3], S[5]])
    V = np.linalg.norm(v)

    # after that, we calculate the field exerted by the magnetic dipole at the location of the charge.
    B = B0 * (1/R)**3 * (np.dot(3*p, r)*r - p)

    # the speed will then be normalised by c, and the relativistic mass will be computed. The velocity (components) is unaffected, however.
    V /= c
    m = m0 / np.sqrt(1 - V**2)

    # finally, we calulate the rates for each of the state variables, and return it from the function.
    # the electromagnetic force on the charged particle is calulated using the Lorentz force law,
    # and the accerlation due to the electromagnetic force is calculated using Newton's second law.
    rate0 = S[1]
    rate1 = (E[0] + (v[1]*B[2] - v[2]*B[1]))*q/m
    rate2 = S[3]
    rate3 = (E[1] + (v[2]*B[0] - v[0]*B[2]))*q/m
    rate4 = S[5]
    rate5 = (E[2] + (v[0]*B[1] - v[1]*B[0]))*q/m
    return [rate0, rate1, rate2, rate3, rate4, rate5]

In [4]:
# Now that everything is defined, the next step is to actually run the simulation, for a total time T, with step size dt.
# S0 is the state vector at t = 0.
T = 300
dt = 0.01
S0 = [3, 1, 0, 0, 0, 1]

sol = solve_ivp(DipoleField, t_span=(0, T), y0=S0, t_eval=np.arange(0, T, dt))

In [5]:
# Finally, we plot the particle's resulting trajectory using plotly's pretty graphing capabilities...
x = sol.y[0]
y = sol.y[2]
z = sol.y[4]

df = pd.DataFrame()

interval = 5
df['x'] = x[::interval]
df['y'] = y[::interval]
df['z'] = z[::interval]
df['time'] = sol.t[::interval]

fig = px.scatter_3d(df, x='x', y="y", z="z", template="plotly_dark", color="time")
fig.update_traces(marker=dict(size=1))
fig.update_layout(scene = dict(xaxis = dict(visible=False), yaxis = dict(visible=False), zaxis =dict(visible=False)))
fig.update(layout_coloraxis_showscale=False)
fig.show()